In [ ]:
# Get Insight data by only replacing the *owner and project name *with appropriate *relative path *of the cloned repository from github

In [1]:
import pandas as pd
from typing import Optional
from github import Github
from github.Repository import Repository
import os
from subprocess import check_output
import subprocess
import time

In [2]:
owner = "facebook"
project = "react"

In [3]:
print("Current directory ",os.getcwd())

Current directory  C:\Users\HASAN\Desktop\react_vue_insight_analysis


In [4]:
dir = "C:/Users/HASAN/Desktop/react_vue_insight_analysis/"+ project
isdir = os.path.isdir(dir)
if isdir:
    print('Repository Already cloned in this directory')
else:
    print("Cloning "+ project + " repository...")
    clone_command = "git clone https://github.com/"+ owner + "/" + project + ".git"
    tmp = check_output(clone_command, shell=True).decode()
    print(tmp)
    print("done")

Repository Already cloned in this directory


In [5]:
os.chdir(dir)
print("Current directory ",os.getcwd())

Current directory  C:\Users\HASAN\Desktop\react_vue_insight_analysis\react


In [6]:
labels = ['Base', 'head', 'files_changed', 'addition', 'deletion', 'Changed_lines', 'total_commit', 'total_contributor']
labels_for_author_commit = ['Base', 'head', 'Author_name', 'commit_count']

In [7]:
csv_results = []
csv_results_for_author_commit = []
class GithubStatisticsGeneration(object):

    def __init__(self, user_name: Optional[str] = None, password: Optional[str] = None,
                 access_token: Optional[str] = None):
        self._user_name: str = user_name
        self._password: str = password
        self._access_token: str = access_token

    def get_tags(self, repository: str):
        _cred: Github = self._authenticate_to_github(self._access_token, self._password, self._user_name)
        _repo: Repository = _cred.get_repo(repository)
        re = _repo.get_releases()

        _tags = list(_repo.get_releases())
        _tags = sorted(_tags, key=lambda x: x.created_at, reverse = True)
        print(len(_tags))       
#         print(_tags[0].published_at)
        for i in range(len(_tags)- 1):
            head = _tags[i].tag_name
            base = _tags[i+1].tag_name
            print(base, head)
#             print(_tags[i].created_at)
            try:
                get_addition_deletion_command = "git diff "+ base + "..." + head + " --shortstat"
                get_number_of_commit_command = "git rev-list --count " + base + ".." + head
                get_number_of_contributor_command = "git shortlog " + base + ".." + head + ' -sn | find /c /v ""'

                print(get_addition_deletion_command)
                addition_deletion = check_output(get_addition_deletion_command, shell=True).decode()
                addition_deletion_list = addition_deletion.split(" ")
                print(addition_deletion_list, len(addition_deletion_list))
    #             print(addition_deletion_list[1], addition_deletion_list[4], addition_deletion_list[6])

                print(get_number_of_commit_command)
                commit_no = check_output(get_number_of_commit_command, shell=True).decode()
                print("commit_no: ", commit_no)

                print(get_number_of_contributor_command)
                contributor_no = check_output(get_number_of_contributor_command, shell=True).decode()
                print("contributor_no: ", contributor_no)

                total_change = int(addition_deletion_list[4])+int(addition_deletion_list[6])
                
                # Get Author individual contribution information
                get_authors_individual_commit_count_command = "git shortlog " + base + ".." + head + ' -sn'
                print(get_authors_individual_commit_count_command)
                authors_individual_commit = check_output(get_authors_individual_commit_count_command, shell=True).decode()
                authors_individual_commit_list = authors_individual_commit.splitlines()
                
                for authors_commit in authors_individual_commit_list:
                #     print(authors_commit)
                    author_commit = authors_commit.split('\t')
                    authors_commit_count = int( author_commit[0].strip() )
                    author_name = author_commit[1]
                #     print(type(author_commit), author_commit, len(author_commit))
                    print("authors_commit_count ", authors_commit_count, "author_name ", author_name)
                    
                    csv_results_for_author_commit.append( [base, head, author_name, authors_commit_count] )
                
                csv_results.append([base, head, addition_deletion_list[1], addition_deletion_list[4], addition_deletion_list[6], total_change, int(commit_no), int(contributor_no) ])
                print("===============================================")
            except:
                print("Exception occured")
                continue   
        print("done")        
        
    @staticmethod
    def _authenticate_to_github(access_token: str, password: str, user_name: str):
        if access_token is None:
            cred: Github = Github(user_name, password)
        else:
            cred: Github = Github(access_token)
        return cred



In [8]:
if __name__ == "__main__":
    token = "Provide your github access token"
    github = GithubStatisticsGeneration(access_token= token)
    
    owner_project = owner + "/" + project
    github.get_tags(owner_project)
    
    result_df = pd.DataFrame(data=csv_results, columns = labels )
    result_df.to_csv ('../' + project + '.csv', sep=",", index=None)
    print(result_df.head(3))
    print("========================================================================\n")
    result_df_for_author_commit = pd.DataFrame(data=csv_results_for_author_commit, columns = labels_for_author_commit )
    result_df_for_author_commit.to_csv ('../' + project + '_authors_individual_contribution.csv', sep=",", index=None)
    print(result_df_for_author_commit.head(3))

94
v16.14.0 v17.0.0
git diff v16.14.0...v17.0.0 --shortstat
['', '24', 'files', 'changed,', '39', 'insertions(+),', '39', 'deletions(-)\n'] 8
git rev-list --count v16.14.0..v17.0.0
commit_no:  1

git shortlog v16.14.0..v17.0.0 -sn | find /c /v ""
contributor_no:  1

git shortlog v16.14.0..v17.0.0 -sn
authors_commit_count  1 author_name  Dan Abramov
0.14.10 v16.14.0
git diff 0.14.10...v16.14.0 --shortstat
[''] 1
git rev-list --count 0.14.10..v16.14.0
commit_no:  0

git shortlog 0.14.10..v16.14.0 -sn | find /c /v ""
Exception occured
v15.7.0 0.14.10
git diff v15.7.0...0.14.10 --shortstat
[''] 1
git rev-list --count v15.7.0..0.14.10
commit_no:  0

git shortlog v15.7.0..0.14.10 -sn | find /c /v ""
Exception occured
v16.13.1 v15.7.0
git diff v16.13.1...v15.7.0 --shortstat
['', '2068', 'files', 'changed,', '461466', 'insertions(+),', '89188', 'deletions(-)\n'] 8
git rev-list --count v16.13.1..v15.7.0
commit_no:  777

git shortlog v16.13.1..v15.7.0 -sn | find /c /v ""
contributor_no:  117

gi

['', '203', 'files', 'changed,', '16923', 'insertions(+),', '4347', 'deletions(-)\n'] 8
git rev-list --count v16.11.0..v16.12.0
commit_no:  48

git shortlog v16.11.0..v16.12.0 -sn | find /c /v ""
contributor_no:  14

git shortlog v16.11.0..v16.12.0 -sn
authors_commit_count  9 author_name  Eli White
authors_commit_count  7 author_name  Dan Abramov
authors_commit_count  6 author_name  Andrew Clark
authors_commit_count  6 author_name  Brian Vaughn
authors_commit_count  6 author_name  Dominic Gannaway
authors_commit_count  5 author_name  Sebastian Markbåge
authors_commit_count  2 author_name  Moti Zilberman
authors_commit_count  1 author_name  Deniz Susman
authors_commit_count  1 author_name  Ives van Hoorne
authors_commit_count  1 author_name  Lea Rosema
authors_commit_count  1 author_name  Mateusz Burzyński
authors_commit_count  1 author_name  Vasilii Cuhar
authors_commit_count  1 author_name  Waseem Dahman
authors_commit_count  1 author_name  Wilco Fiers
v16.10.2 v16.11.0
git diff v16.1

['', '325', 'files', 'changed,', '33409', 'insertions(+),', '19681', 'deletions(-)\n'] 8
git rev-list --count v16.8.6..v16.9.0-alpha.0
commit_no:  140

git shortlog v16.8.6..v16.9.0-alpha.0 -sn | find /c /v ""
contributor_no:  22

git shortlog v16.8.6..v16.9.0-alpha.0 -sn
authors_commit_count  46 author_name  Dan Abramov
authors_commit_count  20 author_name  Dominic Gannaway
authors_commit_count  19 author_name  Andrew Clark
authors_commit_count  11 author_name  Brian Vaughn
authors_commit_count  10 author_name  Sebastian Markbåge
authors_commit_count  7 author_name  Eli White
authors_commit_count  7 author_name  Nicolas Gallagher
authors_commit_count  2 author_name  Behzad Abbasi
authors_commit_count  2 author_name  Brandon Dail
authors_commit_count  2 author_name  Farhad Yasir
authors_commit_count  2 author_name  Nathan Hunzaker
authors_commit_count  2 author_name  Sunil Pai
authors_commit_count  1 author_name  Jared Palmer
authors_commit_count  1 author_name  Josh R
authors_commit_c

['', '132', 'files', 'changed,', '8917', 'insertions(+),', '1519', 'deletions(-)\n'] 8
git rev-list --count v16.6.0..v16.6.1
commit_no:  68

git shortlog v16.6.0..v16.6.1 -sn | find /c /v ""
contributor_no:  22

git shortlog v16.6.0..v16.6.1 -sn
authors_commit_count  20 author_name  Andrew Clark
authors_commit_count  18 author_name  Sophie Alpert
authors_commit_count  8 author_name  Dan Abramov
authors_commit_count  2 author_name  Alex Taylor
authors_commit_count  2 author_name  Brian Vaughn
authors_commit_count  2 author_name  Sebastian Markbåge
authors_commit_count  1 author_name  Caleb Meredith
authors_commit_count  1 author_name  Chris Bianca
authors_commit_count  1 author_name  Jeffrey Zhang
authors_commit_count  1 author_name  Jordan Harband
authors_commit_count  1 author_name  Joseph
authors_commit_count  1 author_name  Keyan Zhang
authors_commit_count  1 author_name  Maksim Markelov
authors_commit_count  1 author_name  Marcelo Jorge Vieira
authors_commit_count  1 author_name  M

contributor_no:  1

git shortlog v16.4.1..v16.4.2 -sn
authors_commit_count  3 author_name  Dan Abramov
v16.4.0 v16.4.1
git diff v16.4.0...v16.4.1 --shortstat
['', '125', 'files', 'changed,', '3931', 'insertions(+),', '2343', 'deletions(-)\n'] 8
git rev-list --count v16.4.0..v16.4.1
commit_no:  46

git shortlog v16.4.0..v16.4.1 -sn | find /c /v ""
contributor_no:  19

git shortlog v16.4.0..v16.4.1 -sn
authors_commit_count  10 author_name  Dan Abramov
authors_commit_count  8 author_name  Flarnie Marchan
authors_commit_count  5 author_name  Brian Vaughn
authors_commit_count  3 author_name  Andrew Clark
authors_commit_count  3 author_name  Nathan Hunzaker
authors_commit_count  2 author_name  Ivan Babak
authors_commit_count  2 author_name  Nathan Quarles
authors_commit_count  2 author_name  Philipp Spieß
authors_commit_count  1 author_name  Daniel Lo Nigro
authors_commit_count  1 author_name  Demian Dekoninck
authors_commit_count  1 author_name  Ende93
authors_commit_count  1 author_name  E

...

['', '167', 'files', 'changed,', '994', 'insertions(+),', '585', 'deletions(-)\n'] 8
git rev-list --count v0.13.2..v0.13.3
commit_no:  31

git shortlog v0.13.2..v0.13.3 -sn | find /c /v ""
contributor_no:  6

git shortlog v0.13.2..v0.13.3 -sn
authors_commit_count  19 author_name  Paul O’Shannessy
authors_commit_count  6 author_name  Sophie Alpert
authors_commit_count  3 author_name  Jim Sproch
authors_commit_count  1 author_name  Max Heiber
authors_commit_count  1 author_name  Neri Marschik
authors_commit_count  1 author_name  Scott Feeney
v0.13.1 v0.13.2
git diff v0.13.1...v0.13.2 --shortstat
['', '146', 'files', 'changed,', '43164', 'insertions(+),', '1740', 'deletions(-)\n'] 8
git rev-list --count v0.13.1..v0.13.2
commit_no:  69

git shortlog v0.13.1..v0.13.2 -sn | find /c /v ""
contributor_no:  6

git shortlog v0.13.1..v0.13.2 -sn
authors_commit_count  40 author_name  Paul O’Shannessy
authors_commit_count  20 author_name  Sophie Alpert
authors_commit_count  4 author_name  Jim Sproc

contributor_no:  5

git shortlog v0.12.1..v0.12.2 -sn
authors_commit_count  33 author_name  Paul O’Shannessy
authors_commit_count  3 author_name  Cheng Lou
authors_commit_count  3 author_name  Vjeux
authors_commit_count  2 author_name  Sophie Alpert
authors_commit_count  2 author_name  Steven Luscher
v0.12.0 v0.12.1
git diff v0.12.0...v0.12.1 --shortstat
['', '26', 'files', 'changed,', '804', 'insertions(+),', '667', 'deletions(-)\n'] 8
git rev-list --count v0.12.0..v0.12.1
commit_no:  19

git shortlog v0.12.0..v0.12.1 -sn | find /c /v ""
contributor_no:  8

git shortlog v0.12.0..v0.12.1 -sn
authors_commit_count  8 author_name  Paul O’Shannessy
authors_commit_count  4 author_name  Jeff Morrison
authors_commit_count  2 author_name  Vjeux
authors_commit_count  1 author_name  Andres Suarez
authors_commit_count  1 author_name  Cheng Lou
authors_commit_count  1 author_name  Gabe Levi
authors_commit_count  1 author_name  Sophie Alpert
authors_commit_count  1 author_name  Zacharias
v0.12.0-rc

['', '26', 'files', 'changed,', '3434', 'insertions(+),', '122', 'deletions(-)\n'] 8
git rev-list --count v0.11.0-rc1..v0.11.0
commit_no:  30

git shortlog v0.11.0-rc1..v0.11.0 -sn | find /c /v ""
contributor_no:  5

git shortlog v0.11.0-rc1..v0.11.0 -sn
authors_commit_count  13 author_name  Sophie Alpert
authors_commit_count  11 author_name  Paul O’Shannessy
authors_commit_count  3 author_name  Cheng Lou
authors_commit_count  2 author_name  Thomas Shaddox
authors_commit_count  1 author_name  Mouad Debbar
v0.10.0 v0.11.0-rc1
git diff v0.10.0...v0.11.0-rc1 --shortstat
['', '282', 'files', 'changed,', '10421', 'insertions(+),', '3738', 'deletions(-)\n'] 8
git rev-list --count v0.10.0..v0.11.0-rc1
commit_no:  504

git shortlog v0.10.0..v0.11.0-rc1 -sn | find /c /v ""
contributor_no:  89

git shortlog v0.10.0..v0.11.0-rc1 -sn
authors_commit_count  122 author_name  Sophie Alpert
authors_commit_count  116 author_name  Paul O’Shannessy
authors_commit_count  59 author_name  Cheng Lou
authors_c

commit_no:  48

git shortlog v0.9.0-rc1..v0.9.0 -sn | find /c /v ""
contributor_no:  10

git shortlog v0.9.0-rc1..v0.9.0 -sn
authors_commit_count  15 author_name  Paul O’Shannessy
authors_commit_count  13 author_name  Pete Hunt
authors_commit_count  13 author_name  Sophie Alpert
authors_commit_count  1 author_name  Andrey Popp
authors_commit_count  1 author_name  Ben Newman
authors_commit_count  1 author_name  Cheng Lou
authors_commit_count  1 author_name  Dustin Getz
authors_commit_count  1 author_name  Josh Duck
authors_commit_count  1 author_name  Martin Andert
authors_commit_count  1 author_name  Petri Lievonen
v0.8.0 v0.9.0-rc1
git diff v0.8.0...v0.9.0-rc1 --shortstat
['', '411', 'files', 'changed,', '18813', 'insertions(+),', '8126', 'deletions(-)\n'] 8
git rev-list --count v0.8.0..v0.9.0-rc1
commit_no:  1012

git shortlog v0.8.0..v0.9.0-rc1 -sn | find /c /v ""
contributor_no:  78

git shortlog v0.8.0..v0.9.0-rc1 -sn
authors_commit_count  173 author_name  Paul O’Shannessy
authors

contributor_no:  17

git shortlog v0.5.1..v0.4.2 -sn
authors_commit_count  23 author_name  Paul O’Shannessy
authors_commit_count  9 author_name  Vjeux
authors_commit_count  3 author_name  Hugo Jobling
authors_commit_count  3 author_name  Sophie Alpert
authors_commit_count  2 author_name  Cheng Lou
authors_commit_count  2 author_name  Clay Allsopp
authors_commit_count  2 author_name  Jakub Malinowski
authors_commit_count  2 author_name  Jamie Wong
authors_commit_count  2 author_name  Keito Uchiyama
authors_commit_count  1 author_name  Brian Rue
authors_commit_count  1 author_name  Connor McSheffrey
authors_commit_count  1 author_name  Daniel Gasienica
authors_commit_count  1 author_name  Kunal Mehta
authors_commit_count  1 author_name  Peter Cottle
authors_commit_count  1 author_name  Stoyan Stefanov
authors_commit_count  1 author_name  Thomas Aylott
authors_commit_count  1 author_name  Timothy Yung
v0.5.0 v0.5.1
git diff v0.5.0...v0.5.1 --shortstat
['', '21', 'files', 'changed,', '1452